In [16]:
import requests
from urllib.parse import quote

In [17]:
BASE_URL = 'https://test-pred-maint-projects.onrender.com/'

In [18]:
url = 'https://test-pred-maint-projects.onrender.com/list_features'
response = requests.get(url)

# Afficher le statut de la réponse
print('Statut de la réponse:', response.status_code)

# Afficher le contenu de la réponse
print('Contenu de la réponse:', response.text)

Statut de la réponse: 200
Contenu de la réponse: ["Type","Air temperature [K]","Process temperature [K]","Rotational speed [rpm]","Torque [Nm]","Tool wear [min]","Difference temperature [K]","Power"]


In [19]:
response_json = response.json()
response_json

['Type',
 'Air temperature [K]',
 'Process temperature [K]',
 'Rotational speed [rpm]',
 'Torque [Nm]',
 'Tool wear [min]',
 'Difference temperature [K]',
 'Power']

In [20]:
class feature_numeric:
    def __init__(self, name:str, min, max):
        self.name = name
        self.min = min
        self.max = max

class feature_list:
    def __init__(self, name:str, values:list):
        self.name = name
        self.values = values

In [45]:
dict_numeric = {}
dict_list = {}

for feature in response_json:
    url2 = BASE_URL+quote(f'features/{feature}')
    response2 = requests.get(url2)
    print(url2)
    print(response2.json())
    if response2.json()['type']=='object':
        dict_list[feature] = feature_list(feature, response2.json()['list'])
    else:
        dict_numeric[feature] = feature_numeric(feature, response2.json()['min'], response2.json()['max'])

https://test-pred-maint-projects.onrender.com/features/Type
{'type': 'object', 'min': 'H', 'max': 'M', 'list': "['M' 'L' 'H']"}
https://test-pred-maint-projects.onrender.com/features/Air%20temperature%20%5BK%5D
{'type': 'float64', 'min': '295.3', 'max': '304.5', 'list': 0}
https://test-pred-maint-projects.onrender.com/features/Process%20temperature%20%5BK%5D
{'type': 'float64', 'min': '305.7', 'max': '313.8', 'list': 0}
https://test-pred-maint-projects.onrender.com/features/Rotational%20speed%20%5Brpm%5D
{'type': 'int64', 'min': '1168', 'max': '2886', 'list': 0}
https://test-pred-maint-projects.onrender.com/features/Torque%20%5BNm%5D
{'type': 'float64', 'min': '3.8', 'max': '76.6', 'list': 0}
https://test-pred-maint-projects.onrender.com/features/Tool%20wear%20%5Bmin%5D
{'type': 'int64', 'min': '0', 'max': '253', 'list': 0}
https://test-pred-maint-projects.onrender.com/features/Difference%20temperature%20%5BK%5D
{'type': 'float64', 'min': '7.599999999999966', 'max': '12.100000000000025

In [44]:
import ast
print(dict_list['Type'].values.replace(" ", ","))
tmp = eval(dict_list['Type'].values.replace(" ", ","))
print(len(tmp))
print(type(tmp))

['M','L','H']
3
<class 'list'>


In [24]:
len(dict_list)

1

In [ ]:
dict_numeric['Power'].min